In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

T_f = 350

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFPSP_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=T_f)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=T_f)
TempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
Nmax = 8000
dt = 0.05
t = 0.0
avg_pc_iterations = 0.0
flog = open('FPSP_avg%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP_avg%i.log' % k, mode='a')

abtempS_prev = TempS.curr.copy()
abtempF_prev = TempF.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    abtempS_prev[:] = abTempS.curr
    abtempF_prev[:] = abTempF.curr
    
    # update fluid interface temperature
    TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=TempF.curr
    )
    
    while True:
        # back up previous interface value
        TempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = 0.5*(CS1 * (abTempS.curr+abtempS_prev) + \
                             CS2 * (abTempF.curr[f2s]+abtempF_prev[f2s]))
        # update solid temperature interface
        solverS.update_temperature_interface(
            tag=interface_tag,
            T=TempS.curr
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid ambient temperature, the new one
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        
        # update residual
        TempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(TempF)
                under_relax.update_solution(TempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(TempF.curr+CF1 * abtempS_prev[s2f] + CF2 * abtempF_prev)
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
        solverS.write()
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=7.
step=1, pc_iterations=3.
step=2, pc_iterations=3.
step=3, pc_iterations=3.
step=4, pc_iterations=3.
step=5, pc_iterations=2.
step=6, pc_iterations=2.
step=7, pc_iterations=2.
step=8, pc_iterations=2.
step=9, pc_iterations=2.
step=10, pc_iterations=2.
step=11, pc_iterations=2.
step=12, pc_iterations=2.
step=13, pc_iterations=2.
step=14, pc_iterations=2.
step=15, pc_iterations=2.
step=16, pc_iterations=2.
step=17, pc_iterations=2.
step=18, pc_iterations=2.
step=19, pc_iterations=2.
step=20, pc_iterations=2.
step=21, pc_iterations=2.
step=22, pc_iterations=2.
step=23, pc_iterations=2.
step=24, pc_iterations=2.
step=25, pc_iterations=2.
step=26, pc_iterations=2.
step=27, pc_iterations=2.
step=28, pc_iterations=2.
step=29, pc_iterations=2.
step=30, pc_iterations=2.
step=31, pc_iterations=2.
step=32, pc_iterations=2.
step=33, pc_iterations=2.
step=34, pc_iterations=2.
step=35, pc_iterations=2.
step=36, pc_iterations=2.
step=37, pc_iterations=2.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
print(theta)
sio.savemat(
    'numeric_res/kFPSP_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.07343554 0.06833621 0.06375296 0.06109062 0.05877194 0.05662288
 0.05471154 0.0529627  0.05126426 0.0497358  0.04830647 0.04696815
 0.0456649  0.04444841 0.04328205 0.04219231 0.04112571 0.04011522
 0.03915784 0.03822499 0.03734371 0.03648885 0.03566959 0.03489143
 0.03413145 0.03340268 0.03269699 0.0320331  0.03139062 0.03077296
 0.03017675 0.02960132 0.02904884 0.02851119 0.02799609 0.02750557
 0.02702454 0.02656173 0.0261129  0.02568811 0.02528021 0.02488561
 0.02450444 0.02413641 0.02378156 0.02344144 0.02311075 0.02279694
 0.02249464 0.02219893 0.02191667 0.02164732 0.02138851 0.02113877
 0.02090046 0.0206756  0.02045858 0.02025237 0.02005354 0.01986783
 0.01969176 0.01952174 0.01936253 0.01921331 0.01907629 0.01894515
 0.01882441 0.01871768 0.01861721 0.01852563 0.01844971 0.0183801
 0.01832341 0.01827413 0.018237   0.01821044 0.01819899 0.01819696
 0.01820743 0.01823464 0.0182723  0.01832676 0.01839613 0.01847851
 0.01857985 0.01870171 0.01883816 0.01899872 0.01918593 0.01939